# Struture Preparation with `pyCHARMM` and `crimm`

In this example, we are going to fetch a structure directly from RCSB by PDB ID. Use `crimm` to build any missing loop, set the protonation state on the protein residues, and load the structure into `CHARMM` directly.

We are using **1A8I** as an example for showing the protonation state and loop building, but the routine works on any other protein or RNA structures. You can try **7ZAP** for loading a structure with both protein and RNA present and **4PTI** for disulfide bond patch.

In [4]:
from crimm.Fetchers import fetch_rcsb
from crimm.Modeller import ParameterLoader, TopologyLoader
from crimm.Modeller.LoopBuilder import ChainLoopBuilder
from crimm.Modeller.TopoFixer import fix_chain
from crimm.StructEntities import Model
import crimm.Adaptors.pyCHARMMAdaptors as pcm_interface
from crimm.Adaptors.PropKaAdaptors import PropKaProtonator

from pycharmm.psf import delete_atoms as pcm_del_atoms
from pycharmm.psf import get_natom as pcm_get_natom
from pycharmm.generate import patch as pcm_patch
from pycharmm.settings import set_verbosity as pcm_set_verbosity

In [2]:
def minimize_chain(chain, sd_nstep, abnr_nstep):
    # load into CHARMM to minimize the structure
    if pcm_get_natom() > 0:
        pcm_del_atoms()
    pcm_interface.load_chain(chain)
    pcm_interface.minimize(sd_nstep=sd_nstep, abnr_nstep=abnr_nstep)
    # Uodate the coordinate in crimm structure
    pcm_interface.sync_coords(chain)

def correct_prot_first_patch(chain, default):
    # PRO and GLY need special treatment when patched at the N-terminus 
    first_resname = chain.residues[0].resname
    if first_resname == 'PRO':
        first_patch = 'PROP'
    elif first_resname == 'GLY':
        first_patch = 'GLYP'
    else:
        first_patch = default
    return first_patch

## Parameter and Topology Loaders
Choose the relavent `rtf` and `prm` loader for protein and RNA. The respective topology and parameter files will also be streamed into `CHARMM`

In [3]:
rtf_loader = {
    'prot': TopologyLoader('protein'),
    'na': TopologyLoader('nucleic')
}
param_loader = {
    'prot': ParameterLoader('protein'),
    'na': ParameterLoader('nucleic')
}

# fill the missing ic table values in the respective rtf
for i, (chain_type, cur_rtf) in enumerate(rtf_loader.items()):
    cur_param = param_loader[chain_type]
    cur_param.fill_ic(cur_rtf)
    # load the respective files into CHARMM as well
    prev_level = pcm_set_verbosity(0)
    pcm_interface.load_topology(cur_rtf, append=bool(i))
    pcm_interface.load_parameters(cur_param, append=bool(i))
    pcm_set_verbosity(prev_level)

## Parameters
parameter used for the preparation routine

In [4]:
pdb_id = '1a8i'
prot_first_patch = 'ACE'
prot_last_patch = 'CT3'
na_first_patch = '5TER'
na_last_patch = '3PHO'
sd_nstep = 300
abnr_nstep = 0
charmm_verbosity_level = 0
pH = 7.4

## Fetch structure from RCSB

In [5]:
structure = fetch_rcsb(
    pdb_id,
    include_solvent=False,
    # any existing hydrogen will be removed and rebuilt later
    include_hydrogens=False,
    first_model_only=True
)
# Show the structure
structure

NGLWidget()

<Structure id=1A8I Models=1>
│
├───<Model id=1 Chains=2>
	│
	├───<Polypeptide(L) id=A Residues=813>
	├──────Description: GLYCOGEN PHOSPHORYLASE B
	│
	├───<Heterogens id=B Molecules=1>
	├──────Description: BETA-D-GLUCOPYRANOSE SPIROHYDANTOIN


As we can see below in the sequence, this structure has missing loops (shown in red). We are going to build the loop with `crimm`.

In [6]:
structure.models[0].chains[0].masked_seq.show()

SRPLSDQEKRKQISVRGLAGVENVTELKKNFNRHLHFTLVKDRNVATPRDYYFALAHTVRDHLVGRWIRTQQHYYEKDPKRIYYLSLEFYMGRTLQNTMVNLALENACDEATYQLGLDMEELEEIEEDAGLGNGGLGRLAACFLDSMATLGLAAYGYGIRYEFGIFNQKICGGWQMEEADDWLRYGNPWEKARPEFTLPVHFYGRVEHTSQGAKWVDTQVVLAMPYDTPVPGYRNNVVNTMRLWSAKAPNDFNLKDFNVGGYIQAVLDRNLAENISRVLYPNDNFFEGKELRLKQEYFVVAATLQDIIRRFKSSKFGCRDPVRTNFDAFPDKVAIQLNDTHPSLAIPELMRVLVDLERLDWDKAWEVTVKTCAYTNHTVIPEALERWPVHLLETLLPRHLQIIYEINQRFLNRVAAAFPGDVDRLRRMSLVEEGAVKRINMAHLCIAGSHAVNGVARIHSEILKKTIFKDFYELEPHKFQNKTNGITPRRWLVLCNPGLAEIIAERIGEEYISDLDQLRKLLSYVDDEAFIRDVAKVKQENKLKFAAYLEREYKVHINPNSLFDVQVKRIHEYKRQLLNCLHVITLYNRIKKEPNKFVVPRTVMIGGKAAPGYHMAKMIIKLITAIGDVVNHDPVVGDRLRVIFLENYRVSLAEKVIPAADLSEQISTAGTEASGTGNMKFMLNGALTIGTMDGANVEMAEEAGEENFFIFGMRVEDVDRLDQRGYNAQEYYDRIPELRQIIEQLSSGFFSPKQPDLFKDIVNMLMHHDRFKVFADYEEYVKCQERVSALYKNPREWTRMVIRNIATSGKFSSDRTIAQYAREIWGVEPSRQRLPAPDEKIP


## Separate Chains by Chain Type
First we need to separate the chain types. Although in this example, we do not have RNA chain, but this routine is built to accommodate both types.

In [7]:
prot_chains = {}
na_chains = {}
# get the first model's id
model_id = structure.models[0].id
# create a new empty model to store chains of interests
new_model = Model(model_id)
for chain in structure[model_id].chains:
    if chain.chain_type == 'Polypeptide(L)':
        prot_chains[chain.id] = chain
    elif chain.chain_type  in ('Polyribonucleotide', 'Polydeoxyribonucleotide'):
        na_chains[chain.id] = chain

## Generate Topology and Loop Building with crimm First

### Protein Chains

In [8]:
for chain_id, chain in prot_chains.items():
    need_minimization = False
    # Missing loop in the chain
    if not chain.is_continuous():
        loop_builder = ChainLoopBuilder(chain)
        # Coordinates of the missing residues will be copied from
        # Alphafold structures
        # only build the loop not the termini
        loop_builder.build_from_alphafold(include_terminal = False)
        chain = loop_builder.get_chain()
        prot_chains[chain_id] = chain
        need_minimization = True
    prot_first_patch = correct_prot_first_patch(chain, default = prot_first_patch)
    rtf_loader['prot'].generate_chain_topology(
        chain,
        first_patch=prot_first_patch, 
        last_patch=prot_last_patch,
        # Coerce any modified residue to canonical residue that it is based on
        coerce=True
    )
    param_loader['prot'].fill_ic(rtf_loader['prot'])
    param_loader['prot'].apply(chain.topo_elements)
    fix_chain(chain)
    if need_minimization:
        # load into CHARMM to minimize the structure
        prev_level = pcm_set_verbosity(charmm_verbosity_level)
        minimize_chain(chain, sd_nstep, abnr_nstep)
        pcm_set_verbosity(prev_level)
    new_model.add(chain)

/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/Superimpose/ChainSuperimposer.py:68: UserWarning: Residues are not identical at model chain: ILE-380 and template chain: LEU-381
  warnings.warn(
/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/Superimpose/ChainSuperimposer.py:68: UserWarning: Residues are not identical at model chain: LLP-680 and template chain: LYS-681
  warnings.warn(
/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/Modeller/TopoLoader.py:331: UserWarning: Residue LLP is not defined in the topology file!
  warnings.warn(
/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/Modeller/TopoLoader.py:436: UserWarning: Coerced Residue LLP to LYS
  warnings.warn(
/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/Modeller/TopoLoader.py:370: UserWarning: Atom N1 from (680, 'LYS') is not defined in the topology file!
  warnings.warn(
/home/truman/.conda/envs/crimm-conda/lib

 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS216 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD - 22  HSE -  0  HSP -  0  ASP - 47  GLU - 64  LYS - 47  TYR - 36
 EPHI: WARNING. bent improper torsion angle is 

far from minimum for;
    IPHI=  925  with deltaPHI=-163.6911 MIN=   0.0000 ATOMS: 5135 5133 5137 5136
 EPHI: WARNING. bent improper torsion angle is 

far from minimum for;
    IPHI=  917  with deltaPHI= 156.3151 MIN=   0.0000 ATOMS: 5095 5083 5097 5096
 EPHI: WARNING. bent improper torsion angle is 

far from minimum for;
    IPHI=  918  with deltaPHI=-123.1358 MIN=   0.0000 ATOMS: 5097 5095 5099 5098
 EPHI: WARNING. bent improper torsion angle is 

far from minimum for;
    IPHI=  917  with deltaPHI=  90.0918 MIN=   0.0000 ATOMS: 5095 5083 5097 5096
 EPHI: WARNING. bent improper torsion angle is 

far from minimum for;
    IPHI=  917  with deltaPHI=  92.9407 MIN=   0.0000 ATOMS: 5095 5083 5097 5096
 EPHI: 

### RNA Chains
DNA chains are not yet supported but will be implemented soon

In [9]:
for chain_id, chain in na_chains.items():
    # Missing loop is very unlikely in nucleotide chains on PDB
    # but if it exsits, an error will be raise
    if not chain.is_continuous():
        raise ValueError(
            f'Nucleotide chain {chain.id} is not continuous, '
            'topology cannot be generated.'
        )
    rtf_loader['na'].generate_chain_topology(
        chain, 
        first_patch=na_first_patch,
        last_patch=na_last_patch,
        coerce=True
    )
    param_loader['na'].fill_ic(rtf_loader['na'])
    param_loader['na'].apply(chain.topo_elements)
    fix_chain(chain)
    new_model.add(chain)

Finally, replace the model with the new model in the structure

In [10]:
new_model.set_connect(structure.models[0].connect_dict)
structure.detach_child(model_id)
structure.add(new_model)

/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/StructEntities/Model.py:112: UserWarning: Residue 680 in chain A has name LYS, not LLP
  warnings.warn(
/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record covale
  warnings.warn(


The loops are built, and now it is ready for protonation state calculation.

In [11]:
structure

NGLWidget()

<Structure id=1A8I Models=1>
│
├───<Model id=1 Chains=1>
	│
	├───<Polypeptide(L) id=A Residues=829>
	├──────Description: GLYCOGEN PHOSPHORYLASE B


## Get Protonation State from the specified pH Value
Note that the protonator accept Model level entity not the structure itself

In [12]:
protonator = PropKaProtonator(rtf_loader['prot'], param_loader['prot'], pH = pH)
protonator.load_model(new_model)
protonator.apply_patches()
if pcm_get_natom() > 0:
    pcm_del_atoms()
for chain in new_model:
    if len(protonator.patches[chain.id]) > 0:
        built_atoms = fix_chain(chain)
    # Also load the chain into CHARMM
    pcm_interface.load_chain(chain)

Unexpected number (12) of atoms in residue GLN   7 A   in conformation 1A
Unexpected number (9) of atoms in residue PRO 835 A   in conformation 1A
/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/Modeller/TopoLoader.py:370: UserWarning: Atom HZ3 from (538, 'LYS') is not defined in the topology file!
  warnings.warn(
/home/truman/.conda/envs/crimm-conda/lib/python3.9/site-packages/crimm/Modeller/TopoLoader.py:370: UserWarning: Atom HZ3 from (608, 'LYS') is not defined in the topology file!
  warnings.warn(



 Message from MAPIC: Atom numbers are changed.

 Message from MAPIC:        829 residues deleted.

 Message from MAPIC:          1 segments deleted.
 DELTIC:     13622 bonds deleted
 DELTIC:     24586 angles deleted
 DELTIC:     36025 dihedrals deleted
 DELTIC:      2393 improper dihedrals deleted
 DELTIC:       829 crossterm maps deleted
 DELTIC:      1542 donors deleted
 DELTIC:      1249 acceptors deleted
  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmplykx9pip
 VOPEN> Attempting to open::/tmp/tmplykx9pip::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --   829 RESIDUES
          GLN GLU LYS ARG LYS GLN ILE SER VAL ARG GLY LEU ALA GLY VAL GLU ASN VAL THR GLU 
          LEU LYS LYS ASN PHE ASN ARG HSD LEU HSD PHE THR LEU VAL LYS ASP ARG ASN VAL ALA 
          THR PRO ARG ASP TYR TYR PHE ALA LEU ALA HSD THR VAL ARG ASP HSD LEU VAL GLY ARG 
          TRP ILE ARG THR GLN GLN HSD TYR TYR GLU LYS ASP PRO LYS ARG ILE TYR TYR L

These protonation patches are identified but not yet loaded into `CHARMM`

In [13]:
protonator.patches

{'A': {123: 'GLUP', 296: 'GLUP', 538: 'LSN', 608: 'LSN', 664: 'GLUP'}}

## Update CHARMM Residues with pyCHARMM patch Command

In [14]:
for chain_id, patch_dict in protonator.patches.items():
    for resid, patch_name in patch_dict.items():
        pcm_patch(patch_name, f'PRO{chain_id} {resid}')

  
 CHARMM>     patch GLUP PROA 123
 ATOM  PROA GLU  123  HE2  ADDED.

 Message from MAPIC: Atom numbers are changed.
 AUTGEN: Autogenerating specified angles and dihedrals.
 AUTOGEN: 24586 angles are removed before regeneration for selected atoms.
 AUTOGEN: 36025 dihedrals are removed before regeneration for selected atoms.
 PATCH: Check angles and dihedrals autogenerated.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        1   Number of residues   =      829
         Number of atoms         =    13471   Number of groups     =     4026
         Number of bonds         =    13623   Number of angles     =    24587
         Number of dihedrals     =    36027   Number of impropers  =     2393
         Number of cross-terms   =      829   Number of autogens   =        0
         Number of HB acceptors  =     1249   Number of HB donors  =     1543
         Number of NB exclusions =        0

Check the generated bonds, angle, dihedrals, etc. They should match between CHARMM and crimm

In [15]:
new_model.chains[0].topo_elements

<TopologyElementContainer for <Polypeptide(L) id=A Residues=829> with bonds=13623, angles=24583, dihedrals=36025, impropers=2393, cmap=0>

## Patch Disulfide Bond
If any disulfide bond exists in the structure, we will patch them in the `CHARMM` structure. However, disulfide bonds have not been fully implemented in `crimm`.

In [16]:
if 'disulf' in structure.models[0].connect_dict:
    for res1, res2 in structure.models[0].connect_dict['disulf']:
        seg1, seg2 = res1['chain'], res2['chain']
        seq1, seq2 = res1['resseq'], res2['resseq']
        patch_arg = f'PRO{seg1} {seq1} PRO{seg2} {seq2}'
        print('DISU', patch_arg)
        pcm_patch('DISU', patch_arg)

## Save the structure as PDB and PSF files

In [17]:
from pycharmm import write

In [18]:
write.coor_pdb(f'{pdb_id}.pdb')
write.psf_card(f'{pdb_id}.psf')

  
 CHARMM>     write name 1a8i.pdb -
 CHARMM>     coor pdb
 VOPEN> Attempting to open::1a8i.pdb::
 RDTITL>  
 RDTITL> No title read.
  Write CHARMM-pdb format
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     write name 1a8i.psf -
 CHARMM>     psf card
 VOPEN> Attempting to open::1a8i.psf::
 RDTITL>  
 RDTITL> No title read.
 VCLOSE: Closing unit   91 with status "KEEP"
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  


## Preperation Script with Command Line Interface
The routine above is written as a function in the `load_pdbid_in_charmm.py`. It can also be invoke from command line with
```bash
python -m crimm.Adaptors.charmm_struct_prep 7zap
```
or use it as a function in any python script.

In [1]:
from crimm.Adaptors.charmm_struct_prep import load_pdbid_in_charmm

In [2]:
pdb_id = '7zap'
structure = load_pdbid_in_charmm(
        pdb_id,
        pH = 7.4,
        prot_first_patch = 'ACE',
        prot_last_patch = 'CT3',
        na_first_patch = '5TER',
        na_last_patch = '3PHO',
        sd_nstep = 300,
        abnr_nstep = 0,
        charmm_verbosity_level = 0,
    )

/home/truman/crimm/crimm/Modeller/ParamLoader.py:167: UserWarning: No cmap found in <Polypeptide(L) id=A Residues=97>.
  warnings.warn(
/home/truman/crimm/crimm/Modeller/ParamLoader.py:167: UserWarning: No cmap found in <Polyribonucleotide id=B Residues=28>.
  warnings.warn(
Unexpected number (11) of atoms in residue MET   1 A   in conformation 1A
Unexpected number (7) of atoms in residue ALA  97 A   in conformation 1A


  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmp125p9vvf
 VOPEN> Attempting to open::/tmp/tmp125p9vvf::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --    97 RESIDUES
          MET ILE ASP ASN LEU THR PRO GLU GLU ARG ASP ALA ARG THR VAL PHE CYS MET GLN LEU 
          ALA ALA ARG ILE ARG PRO ARG ASP LEU GLU GLU PHE PHE SER THR VAL GLY LYS VAL ARG 
          ASP VAL ARG MET ILE SER ASP ARG ASN SER ARG ARG SER LYS GLY ILE ALA TYR VAL GLU 
          PHE VAL ASP VAL SER SER VAL PRO LEU ALA ILE GLY LEU THR GLY GLN ARG VAL LEU GLY 
          VAL PRO ILE ILE VAL GLN ALA SER GLN ALA GLU LYS ASN ARG ALA ALA ALA 
 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS 16 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD -  0  HSE -  0  HSP -  0  ASP -  6  GLU -  6  LYS -  3  TYR -  1
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 THE PATCH 'ACE' WILL BE US

In [3]:
structure

NGLWidget()

<Structure id=7ZAP Models=1>
│
├───<Model id=1 Chains=2>
	│
	├───<Polypeptide(L) id=A Residues=97>
	├──────Description: RNA-binding protein 39
	│
	├───<Polyribonucleotide id=B Residues=28>
	├──────Description: U1 snRNA SL3
